# Lab 5. LangChain

- 단축키
> Ctrl + Enter: 해당 셀 실행 <br>
> Shift + Enter: 해당 셀 실행 후, 다음 셀로 이동 <br>
> **셀 클릭 후,** <br>
> A: 위 셀 생성 <br>
> B: 아래 셀 생성 <br>
> D + D: 셀 삭제

## Table of Contents
- Environment setup
- Document Summarization
- Retrieval-Augmented Generation (RAG)
- Chatbot

## Environment setup

In [ ]:
# API key가 제대로 생성되었는지 확인
# 마지막 YOUR_API_KEY를 생성된 API_KEY로 바꾸어 확인
!curl \
  -H 'Content-Type: application/json' \
  -d '{"contents":[{"parts":[{"text":"Write a story about a magic backpack"}]}]}' \
  -X POST https://generativelanguage.googleapis.com/v1beta/models/gemini-pro:generateContent?key=YOUR_API_KEY

In [ ]:
# 필요한 packages를 설치
!pip install langchain==0.1.9
!pip install langchain_google_genai==0.0.9
!pip install langchain-community==0.0.24
!pip install langchainhub==0.1.15
!pip install tiktoken==0.6.0
!pip install chromadb==0.4.24

In [ ]:
# API_KEY 저장
GOOGLE_API_KEY = "YOUR_API_KEY"

## Document Summarization

In [ ]:
from langchain_google_genai import GoogleGenerativeAI
from langchain.prompts import PromptTemplate
from langchain.chains.llm import LLMChain
from langchain.chains.combine_documents.stuff import StuffDocumentsChain

# 현재(24.02.29) 사용 가능 모델: 'models/text-bison-001'(요청 90/분), 'models/gemini-pro' (요청 60/분)
llm = GoogleGenerativeAI(model='models/text-bison-001', google_api_key=GOOGLE_API_KEY)

Summarize a couple of sentences

In [ ]:
content = """
TEXT:
The giant panda (Ailuropoda melanoleuca), sometimes called a panda bear or simply panda, is a bear species endemic to China.
It is characterized by its bold black-and-white coat and rotund body.
The name "giant panda" is sometimes used to distinguish it from the red panda, a neighboring musteloid.
Though it belongs to the order Carnivora, the giant panda is a folivore, with bamboo shoots and leaves making up more than 99% of its diet.
Giant pandas in the wild occasionally eat other grasses, wild tubers, or even meat in the form of birds, rodents, or carrion.
In captivity, they may receive honey, eggs, fish, yams, shrub leaves, oranges, or bananas along with specially prepared food."""

In [ ]:
# string 프롬프트를 위한 템플릿(template)을 생성, 템플릿은 f-strings(default)나 jinja2로 포맷팅(formatting)이 가능

# 1. PromptTemplate.from_template()을 사용하는 방법 (recommended)
prompt = PromptTemplate.from_template("Please provide a summary of the following text in 2 lines. {content}")

# 2. PromptTemplate 생성자를 사용하는 방법
# prompt = PromptTemplate(
#     input_variables=["content"],
#     template="Please provide a summary of the following text in 2 lines. {content}"
# )

In [ ]:
# LLM chain을 설정

# 1. LLMChain 방법을 사용
# llm_chain = LLMChain(llm=llm, prompt=prompt)
# llm_chain.invoke({'content': content})

# 2. LCEL을 사용 (recommended)
llm_chain = prompt | llm
llm_chain.invoke({'content': content})

Summarize a document

In [ ]:
from langchain.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
docs = loader.load()

In [ ]:
docs

In [ ]:
prompt_template = """Write a concise summary of the following:
"{context}"
CONCISE SUMMARY:"""

prompt = PromptTemplate.from_template(prompt_template)

In [ ]:
llm = GoogleGenerativeAI(model='models/text-bison-001',
                         google_api_key=GOOGLE_API_KEY)

In [ ]:
from langchain.chains.combine_documents.stuff import create_stuff_documents_chain

chain = create_stuff_documents_chain(llm=llm, prompt=prompt)
chain.invoke({'context': docs})

### Exercise
**None 부분을 채워주세요!**

In [ ]:
from langchain.chains.combine_documents.stuff import create_stuff_documents_chain

loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
docs = loader.load()

prompt_template = None

prompt = None

llm = GoogleGenerativeAI(model='models/text-bison-001',
                         google_api_key=GOOGLE_API_KEY)

chain = None
chain.invoke(None)

In [ ]:
#@title 정답코드

from langchain.chains.combine_documents.stuff import create_stuff_documents_chain

loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
docs = loader.load()

prompt_template = """Summarize the following blog post like a news article title and news abstract:
"{context}"
News title and abstract:"""

prompt = PromptTemplate.from_template(prompt_template)

llm = GoogleGenerativeAI(model='models/text-bison-001',
                         google_api_key=GOOGLE_API_KEY)

chain = create_stuff_documents_chain(llm=llm, prompt=prompt)
chain.invoke({'context': docs})

### Exercise
**None 부분을 채워주세요!**

In [ ]:
loader = WebBaseLoader("https://www.sciencealert.com/one-of-the-fastest-meteor-showers-of-the-year-is-peaking-this-weekend-look-up")
docs = loader.load()

prompt_template = None

prompt = None

llm = GoogleGenerativeAI(model='models/text-bison-001',
                         google_api_key=GOOGLE_API_KEY)

chain = None
chain.invoke(None)

In [ ]:
#@title 정답코드

loader = WebBaseLoader("https://www.sciencealert.com/one-of-the-fastest-meteor-showers-of-the-year-is-peaking-this-weekend-look-up")
docs = loader.load()

prompt_template = """Summarize the following news article in a few sentences and extract the hashtags:
"{context}"
Summarization and Hashtags:"""

prompt = PromptTemplate.from_template(prompt_template)

llm = GoogleGenerativeAI(model='models/text-bison-001',
                         google_api_key=GOOGLE_API_KEY)

chain = create_stuff_documents_chain(llm=llm, prompt=prompt)
chain.invoke({'context': docs})

## Retrieval-Augmented Generation (RAG)

### Similarity Search

In [ ]:
import bs4
from langchain.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain_google_genai.embeddings import GoogleGenerativeAIEmbeddings

In [ ]:
# 웹페이지(문서)를 로드
loader = WebBaseLoader(web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",), bs_kwargs=dict(parse_only=bs4.SoupStrainer(class_=("post-content", "post-title", "post-header"))),)

raw_documents = loader.load()

In [ ]:
# 문서를 청크로 분리
text_splitter = RecursiveCharacterTextSplitter(
  chunk_size=1000, # Maximum size of chunks to return
  chunk_overlap=200 # Overlap in characters between chunks
)
splits = text_splitter.split_documents(raw_documents)

In [ ]:
# 각 청크를 임베딩하고 이를 벡터 스토어에 저장
vectorstore = Chroma.from_documents(
  documents=splits,
  embedding=GoogleGenerativeAIEmbeddings(model='models/embedding-001', google_api_key=GOOGLE_API_KEY)
)

In [ ]:
# 벡터 스토어에 저장되어 있는 딕셔너리 키 값을 출력
print(vectorstore.get().keys())

In [ ]:
from pprint import pprint

# 특정 ids의 값을 출력
id = vectorstore.get()["ids"][0]
print(f'ID: {id}')
pprint(vectorstore.get(ids=id))

In [ ]:
# 벡터 스토어에 저장되어 있는 문서(청크) 두 개를 출력
for i in range(2):
  print(f'** Chunk {i} **')
  print(vectorstore.get()['documents'][i])
  print()

In [ ]:
query = "What are the types of memory?"

docs = vectorstore.similarity_search(query)

print("** Document #1 **")
print(docs[0].page_content)

print("\n** Document #2 **")
print(docs[1].page_content)

In [ ]:
query = "What are the types of memory?"
query_embedding = GoogleGenerativeAIEmbeddings(model='models/embedding-001', google_api_key=GOOGLE_API_KEY).embed_query(query)

docs = vectorstore.similarity_search_by_vector(query_embedding)

print("** Document #1 **")
print(docs[0].page_content)

print("\n** Document #2 **")
print(docs[1].page_content)

#### Exercise
**None 부분을 채워주세요!**

In [ ]:
# 위 셀에서 생성한 vectorstore 삭제
vectorstore.delete_collection()

In [ ]:
loader = WebBaseLoader("https://www.sciencealert.com/one-of-the-fastest-meteor-showers-of-the-year-is-peaking-this-weekend-look-up")
docs = loader.load()

text_splitter = None
splits = None

vectorstore = None

In [ ]:
query = "What is the Leonid meteor shower?"
# query = "When is this event occur?"
retrieved_docs = None

print(retrieved_docs[0].page_content)

In [ ]:
#@title 정답코드
loader = WebBaseLoader("https://www.sciencealert.com/one-of-the-fastest-meteor-showers-of-the-year-is-peaking-this-weekend-look-up")
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(
  chunk_size=400, # Maximum size of chunks to return
  chunk_overlap=100 # Overlap in characters between chunks
)
splits = text_splitter.split_documents(docs)

vectorstore = Chroma.from_documents(
  documents=splits,
  embedding=GoogleGenerativeAIEmbeddings(model='models/embedding-001', google_api_key=GOOGLE_API_KEY)
)

query = "What is the Leonid meteor shower?"
# query = "When is this event occur?"
retrieved_docs = vectorstore.similarity_search(query)

print(retrieved_docs[0].page_content)

## Web-based Retrieval

In [ ]:
import bs4

from langchain.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain_google_genai.embeddings import GoogleGenerativeAIEmbeddings

from langchain import hub
from langchain.schema import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
from langchain_google_genai import GoogleGenerativeAI

In [ ]:
# 위 셀에서 생성한 vectorstore 삭제
vectorstore.delete_collection()

In [ ]:
# 웹페이지(문서)를 로드
loader = WebBaseLoader(web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",), bs_kwargs=dict(parse_only=bs4.SoupStrainer(class_=("post-content", "post-title", "post-header"))),)
raw_documents = loader.load()

# 문서를 청크로 분리
text_splitter = RecursiveCharacterTextSplitter(
  chunk_size=1000, # Maximum size of chunks to return
  chunk_overlap=200 # Overlap in characters between chunks
)
splits = text_splitter.split_documents(raw_documents)

# 각 청크를 임베딩하고 이를 벡터 스토어에 저장
vectorstore = Chroma.from_documents(
  documents=splits,
  embedding=GoogleGenerativeAIEmbeddings(model='models/embedding-001', google_api_key=GOOGLE_API_KEY)
)

In [ ]:
# 리트리버 정의
retriever = vectorstore.as_retriever(
  search_type="similarity",
  search_kwargs={"k": 6}
)

retrieved_docs = retriever.get_relevant_documents(
  "What is the MIPS?"
)

In [ ]:
# 리트리브 된 문서의 개수를 출력
print(len(retrieved_docs))
print()

# 첫 리트리브된 문서의 문서 내용을 출력
print(retrieved_docs[0].page_content)

In [ ]:
# LLM 모델을 정의
llm = GoogleGenerativeAI(model='models/gemini-pro', google_api_key=GOOGLE_API_KEY)

# 사전 정의된 프롬프트를 가져오기
prompt = hub.pull("rlm/rag-prompt")

In [ ]:
# 사전 정의된 프롬프트 내용 확인
# https://smith.langchain.com/hub/rlm/rag-prompt
hub.pull("rlm/rag-prompt")

In [ ]:
# 여러 개의 문서가 있을 때 이를 string으로 이어주는 함수
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# 체인 생성
rag_chain = (
  {"context": retriever | format_docs,
   "question": RunnablePassthrough()}
  | prompt
  | llm
  | StrOutputParser()
)

In [ ]:
rag_chain

In [ ]:
for chunk in rag_chain.stream("What is the MIPS?"):
    print(chunk, end="", flush=True)

Stream과 Invoke 비교

In [ ]:
# stream
for chunk in rag_chain.stream("What is the MIPS?"):
    print(chunk, end="|", flush=True)

In [ ]:
# invoke
rag_chain.invoke("What is the MIPS?")

#### Exercise
**None 부분을 채워주세요!**

In [ ]:
# 위 셀에서 생성한 vectorstore 삭제
vectorstore.delete_collection()

In [ ]:
from langchain import hub

# Indexing
loader = WebBaseLoader("https://www.sciencealert.com/one-of-the-fastest-meteor-showers-of-the-year-is-peaking-this-weekend-look-up")
docs = loader.load()

text_splitter = None
splits = None

vectorstore = None

In [ ]:
# Retrieve
# search type: similarity, search_kwargs: {"k": 2}
retriever = None

# LLM 모델을 정의
llm = GoogleGenerativeAI(model='models/gemini-pro', google_api_key=GOOGLE_API_KEY)

# 사전 정의된 프롬프트를 가져오기
prompt = None

# 여러 개의 문서가 있을 때 이를 string으로 이어주는 함수
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [ ]:
# 체인 생성
rag_chain = None

In [ ]:
for chunk in rag_chain.stream("What is the Leonid meteor shower?"):
    print(chunk, end="", flush=True)

In [ ]:
for chunk in rag_chain.stream("When is this event occur?"):
    print(chunk, end="", flush=True)

In [ ]:
#@title 정답코드

from langchain import hub

# Indexing
loader = WebBaseLoader("https://www.sciencealert.com/one-of-the-fastest-meteor-showers-of-the-year-is-peaking-this-weekend-look-up")
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(
  chunk_size=400, # Maximum size of chunks to return
  chunk_overlap=100 # Overlap in characters between chunks
)
splits = text_splitter.split_documents(docs)

vectorstore = Chroma.from_documents(
  documents=splits,
  embedding=GoogleGenerativeAIEmbeddings(model='models/embedding-001', google_api_key=GOOGLE_API_KEY)
)

# Retrieve
retriever = vectorstore.as_retriever(
  search_type="similarity",
  search_kwargs={"k": 2}
)

# LLM 모델을 정의
llm = GoogleGenerativeAI(model='models/gemini-pro', google_api_key=GOOGLE_API_KEY)

# 사전 정의된 프롬프트를 가져오기
prompt = hub.pull("rlm/rag-prompt")

# 여러 개의 문서가 있을 때 이를 string으로 이어주는 함수
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# 체인 생성
rag_chain = (
  {"context": retriever | format_docs,
   "question": RunnablePassthrough()}
  | prompt
  | llm
  | StrOutputParser()
)

for chunk in rag_chain.stream("What is the Leonid meteor shower?"):
    print(chunk, end="", flush=True)

# for chunk in rag_chain.stream("When is this event occur?"):
#   print(chunk, end="", flush=True)

## Chatbot

Memory

In [ ]:
from langchain.memory import ChatMessageHistory

memory = ChatMessageHistory()

memory.add_user_message("hi!")
memory.add_ai_message("whats up?")

print(memory.messages)

In [ ]:
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_google_genai import GoogleGenerativeAI

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant.",
        ),
        HumanMessage(content="What is 3+5?"),
        AIMessage(content="It is 8."),
        HumanMessage(content="What is 4x3?"),
        AIMessage(content="It is 12."),
        HumanMessage(content="Add above equations.")
    ]
)

# 간단한 체인 정의
llm = GoogleGenerativeAI(model='models/gemini-pro', google_api_key=GOOGLE_API_KEY)
chain = prompt | llm

chain.invoke({'input': ''})

In [ ]:
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | llm
chain.invoke({'messages': [
    HumanMessage(content="What is 3+5?"),
    AIMessage(content="It is 8."),
    HumanMessage(content="What is 4x3?"),
    AIMessage(content="It is 12."),
    HumanMessage(content="Add above equations.")
]})

In [ ]:
from langchain.memory import ChatMessageHistory

memory = ChatMessageHistory()

memory.add_user_message("What is 3+5?")
memory.add_ai_message("It is 8.")
memory.add_user_message("What is 4x3?")
memory.add_ai_message("It is 12.")
memory.add_user_message("Add above equations.")

chain = prompt | llm
response = chain.invoke({'messages': memory.messages})
memory.add_ai_message(response)

print(response)
print(memory.messages)

### Exercise
**None 부분을 채워주세요!**

In [ ]:
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ChatMessageHistory
from langchain_google_genai import GoogleGenerativeAI

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are helpful assistant. And you always say 'Ho-Ho-Ho.' after you answer my question.",
        ),
        MessagesPlaceholder(variable_name="messages")
    ]
)

memory = None

llm = GoogleGenerativeAI(model='models/gemini-pro', google_api_key=GOOGLE_API_KEY)
chain = None

In [ ]:
while True:
  input_ = input("User: ")
  if input_ == 'quit':
    print("\n**Message history**")
    for m in memory.messages:
      print(m.content)
    break
  # memory에 user의 message를 추가
  None
  response = None
  # memory에 ai의 message를 추가
  None
  print(response)

In [ ]:
#@title 정답코드

from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ChatMessageHistory
from langchain_google_genai import GoogleGenerativeAI

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are helpful assistant. And you always say 'Ho-Ho-Ho.' after you answer my question.",
        ),
        MessagesPlaceholder(variable_name="messages")
    ]
)

memory = ChatMessageHistory()

llm = GoogleGenerativeAI(model='models/gemini-pro', google_api_key=GOOGLE_API_KEY)
chain = prompt | llm

while True:
  input_ = input("User: ")
  if input_ == 'quit':
    print("\n**Message history**")
    for m in memory.messages:
      print(m.content)
    break
  memory.add_user_message(input_)
  response = chain.invoke({'messages': memory.messages})
  memory.add_ai_message(response)
  print(response)

Retrieval

In [ ]:
# 위 셀에서 생성한 vectorstore 삭제
vectorstore.delete_collection()

In [ ]:
# 웹페이지(문서)를 로드
loader = WebBaseLoader(web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",), bs_kwargs=dict(parse_only=bs4.SoupStrainer(class_=("post-content", "post-title", "post-header"))),)
raw_documents = loader.load()

# 문서를 청크로 분리
text_splitter = RecursiveCharacterTextSplitter(
  chunk_size=1000, # Maximum size of chunks to return
  chunk_overlap=200 # Overlap in characters between chunks
)
splits = text_splitter.split_documents(raw_documents)

# 각 청크를 임베딩하고 이를 벡터 스토어에 저장
vectorstore = Chroma.from_documents(
  documents=splits,
  embedding=GoogleGenerativeAIEmbeddings(model='models/embedding-001', google_api_key=GOOGLE_API_KEY)
)

# 문서를 검색
retriever = vectorstore.as_retriever(search_kwargs={"k": 4})
docs = retriever.invoke("What are the types of memory?")

for i in range(len(docs)):
  print(docs[i])

In [ ]:
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_google_genai import GoogleGenerativeAI

SYSTEM_TEMPLATE = """
Answer the user's questions based on the below context.
If the context doesn't contain any relevant information to the question, don't make something up and just say "I don't know":

<context>
{context}
</context>
"""

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            SYSTEM_TEMPLATE,
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

llm = GoogleGenerativeAI(model='models/gemini-pro', google_api_key=GOOGLE_API_KEY)
document_chain = create_stuff_documents_chain(llm, prompt)

In [ ]:
from langchain_core.messages import HumanMessage
from langchain.memory import ChatMessageHistory

memory = ChatMessageHistory()
memory.add_user_message("Explain about short-term memory.")

document_chain.invoke(
    {
        "context": docs,
        "messages": memory.messages,
    }
)

In [ ]:
from langchain_core.messages import HumanMessage
from langchain.memory import ChatMessageHistory

memory = ChatMessageHistory()
memory.add_user_message("Explain about short-term memory.")

document_chain.invoke(
    {
        "context": [], # 검색 결과를 추가하지 않은 경우
        "messages": memory.messages,
    }
)

### Exercise
**None 부분을 채워주세요!**

In [ ]:
# 위 셀에서 생성한 vectorstore 삭제
vectorstore.delete_collection()

In [ ]:
# Retriever와 LLM chain에서 사용됨
HUMAN_MESSAGE = "Tell me about iPhone 16."

# Load Webpage
loader = WebBaseLoader("https://www.macrumors.com/2024/03/20/iphone-16-ultra-thin-bezels-rumor/")
data = loader.load()

# Split the text into chunks
text_splitter = None
all_splits = None

In [ ]:
# Create vector store
vectorstore = None

# Initialize LLM and retriever, k=20
retriever = None

In [ ]:
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_google_genai import GoogleGenerativeAI

SYSTEM_TEMPLATE = """
Answer the user's questions based on the below context.
If the context doesn't contain any relevant information to the question, don't make something up and just say "I don't know":

<context>
{context}
</context>
"""

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            SYSTEM_TEMPLATE,
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

llm = GoogleGenerativeAI(model='models/gemini-pro', google_api_key=GOOGLE_API_KEY)
document_chain = create_stuff_documents_chain(llm, prompt)

In [ ]:
from langchain_core.messages import HumanMessage
from langchain.memory import ChatMessageHistory

memory = ChatMessageHistory()
memory.add_user_message(None)

document_chain.invoke(
    {
        "context": docs,
        "messages": memory.messages,
    }
)

In [ ]:
#@title 정답코드

# Retriever와 LLM chain에서 사용됨
HUMAN_MESSAGE = "Tell me about iPhone 16."

# Load Webpage
loader = WebBaseLoader("https://www.macrumors.com/2024/03/20/iphone-16-ultra-thin-bezels-rumor/")
data = loader.load()

# Split the text into chunks
text_splitter = RecursiveCharacterTextSplitter(
  chunk_size=1000,
  chunk_overlap=200
)
all_splits = text_splitter.split_documents(data)

# Create vector store
vectorstore = Chroma.from_documents(
  documents=all_splits,
  embedding=GoogleGenerativeAIEmbeddings(model='models/embedding-001', google_api_key=GOOGLE_API_KEY)
)

# Initialize LLM and retriever
retriever = vectorstore.as_retriever(search_kwargs={"k": 10})
# Search documents
docs = retriever.invoke(HUMAN_MESSAGE)

from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_google_genai import GoogleGenerativeAI

SYSTEM_TEMPLATE = """
Answer the user's questions based on the below context.

<context>
{context}
</context>
"""

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            SYSTEM_TEMPLATE,
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

llm = GoogleGenerativeAI(model='models/gemini-pro', google_api_key=GOOGLE_API_KEY)
document_chain = create_stuff_documents_chain(llm, prompt)

from langchain_core.messages import HumanMessage
from langchain.memory import ChatMessageHistory

memory = ChatMessageHistory()
memory.add_user_message(HUMAN_MESSAGE)

document_chain.invoke(
    {
        "context": docs,
        "messages": memory.messages,
    }
)